In [ ]:
train_size = 47500

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [ ]:
np.set_printoptions(precision=9)
np.set_printoptions(linewidth=np.inf)

In [ ]:
X = np.load('../../X_train.npz')['arr_0']
Y = np.load('../../Y_train.npz')['arr_0']
TX = np.load('../../X_test.npz')['arr_0']

In [ ]:
XTX = np.concatenate((X, TX), axis=0)

In [ ]:
X, TX = None, None

In [ ]:
XTX_square = XTX ** 2
XTX_cumsum = np.concatenate((np.cumsum(XTX[:, :5000], axis=1), np.cumsum(XTX[:, 5000:], axis=1)), axis=1)
XTX_all = np.concatenate((XTX, XTX_square, XTX_cumsum), axis=1)

In [ ]:
XTX, XTX_square, XTX_cumsum = None, None, None, None

In [ ]:
XTX_all.shape

In [ ]:
X, X_test = XTX_all[:train_size], XTX_all[train_size:]
print(X.shape, X_test.shape)

In [ ]:
# calculate type 1 error
def err1(y, y_pred):
    return np.sum(1.0 * np.abs(y_pred - y)) / len(y_pred)

# calculate type 2 errr
def err2(y, y_pred):
    return np.sum(np.abs(y_pred - y) / y) / len(y_pred)

In [ ]:
scorer = {'t1':make_scorer(err1, greater_is_better=False), 't2':make_scorer(err2, greater_is_better=False)}

In [ ]:
# select best params
y_id = 0
idx = np.random.permutation(47500)[:1000]
X_tmp, y_tmp = X[idx], Y[idx]
params = {'boosting_type':['gbdt', 'dart'],
          'num_leaves':[31, 511],
          'learning_rate':[0.05],
          'n_estimators':[100],
          'n_jobs':[23]}

t1_err = np.Inf
t2_err = np.Inf
t1_best_params = {}
t2_best_params = {}
keys, values = zip(*params.items())
for v in itertools.product(*values):
    param = dict(zip(keys, v))
    print(param)
    t1_errr = 0
    t2_errr = 0
    kf = KFold(n_splits=5, shuffle=False)
    for train_idx, val_idx in kf.split(X_tmp):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = lgb.LGBMRegressor(**param)
        model.fit(X_train, y_train)
        t1_errr += err1(model.predict(X_val), y_val)
        t2_errr += err2(model.predict(X_val), y_val)
    
    if t1_errr / 5 < t1_err:
        t1_best_params = param
        t1_err = t1_errr / 5
    if t2_errr / 5 < t2_err:
        t2_best_params = param
        t2_err = t2_errr / 5

In [ ]:
ty = []
vy = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = LinearRegression(t1_best_params)
        model.fit(X_train, y_train)
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))

        print(err1(model.predict(X_train), y_train), err2(model.predict(X_train), y_train))
        print(err1(tvy[-1], y_val), err2(tvy[-1], y_val))
        
        X_train, X_val, y_train, y_val = None, None, None, None
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t1_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t1_val.csv', index=False, header=False)

In [ ]:
ty = []
vy = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = LinearRegression(t2_best_params)
        model.fit(X_train, y_train)
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
        
        print(err1(model.predict(X_train), y_train), err2(model.predict(X_train), y_train))
        print(err1(tvy[-1], y_val), err2(tvy[-1], y_val))        
        
        X_train, X_val, y_train, y_val = None, None, None, None
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t2_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t2_val.csv', index=False, header=False)